# Moving to Py3K
There are a fair number of differences between Python 2.7 and Py3k. Some of them we have encountered already through the `from __future__ import ...`, and a few through `six`. A quick refresher:

* `print` is now a function (`from __future__ import print_function`)

* Division between integers results in a float (`from __future__ import division`). Use `//` to retain old behavior.

In [ ]:
%%python2
from __future__ import print_function
print(1 / 2)
print(1 / 2.0)
print(1 // 2)
print(1 // 2.0)

* `range(5)` returns a range object, not a list. Range objects behave a lot like a list. Effectively, only thing you can't do with it that you can do with a list is mutate it.

In [ ]:
%%python2
from __future__ import print_function
a = range(5)
print(a)
a[3] = 1000
print(a)

* String literals are now always unicode (`from __future__ import unicode_literals`). This can cause odd behavior in some places when mixing old data with new code (or vice versa). Some data loaders may return a "bytes object" instead of a "text object" (which is what strings are nowadays).

In [ ]:
%%python2
from __future__ import print_function
some_text_data = b"bar"  # regular string in py2, bytes object in py3
if some_text_data == "bar":
    print("They are the same!")
else:
    print("They are not the same!")

* floats now round-trip through text representation

In [ ]:
%%python2
from __future__ import print_function
print(float(str(1000.0123456789)) == 1000.0123456789)

* Can't compare or sort mixed types

In [ ]:
%%python2
from __future__ import print_function
print(sorted([4, '2', 3]))

For a fuller examination of the kinds of changes to expect, see https://portingguide.readthedocs.io/

# Where are the .pyc files?
You are probably used to seeing .pyc files appearing in your directories, shadowing your python modules whenever you import them. In Py3k, the .pyc files are placed in a `__pycache__` directory.

# Numerical literal tweak (3.6)
You can now use as many underscores whereever you like for writing out a number. Typically, one would use it for thousands groupings, but it can be done anywhere. Note, not all editors properly recognize this and so you may get funky syntax coloring. This can also be done for hexadecimals and octals.

In [ ]:
a_num = 100_000.5
print(a_num)

In [ ]:
a_num = 0.000_123_05
print(a_num)

This is also recognized for string to numbers conversion:

In [ ]:
a_num = float("10_000.888_88")
print(a_num)

And, you can even have it included in text output for machine-readable output that is easier for humans to read as well.

In [ ]:
a_num = 10000.88888
str_comma = "{a_num:,}".format(a_num=a_num)
str_underscore = "{a_num:_}".format(a_num=a_num)
print(str_comma)
print(str_underscore)

In [ ]:
float(str_underscore)

# f-strings (3.6)
This gives you a feature that some have been used to having in languages like Perl and Bash. If you prepend an `f` to a quoted string, then it is basically like doing `.format(**all_the_things)` to that string automatically.

In [ ]:
name = "Bob"
age = 23
sentence = f"{name} is {age} years old"
print(sentence)

## Simple statement evaluation
This sort of stuff is available *only* in f-strings, not in regular strings that are being `.format()`-ed.

In [ ]:
print(f"{name} will be {age + 5} years old in 5 years")

## debug f-string (3.8)
Nifty convenience feature just added in python 3.8.

In [ ]:
print(f"{name=}  {age=}")

# Optional type annotations (3.0)
In python, you don't need to specify that a variable is a list or an integer, unlike in languages like C/C++. This allows us a lot of ease in just simply writing algorithms rather than focusing on ensuring we got all of the types exactly right. However, there are times when it would be good to know what types are expected where. So, Python introduced syntax for type annotations (a.k.a., type hints).

In [ ]:
def my_add(a: int, b: int) -> int:
    return a + b

Note, this is *not* enforced. It is strictly considered to be an annotation.

In [ ]:
my_add("foo", "bar")

But, one can use a python linter to try and detect mistakes in a codebase using this feature. This feature is still being developed and slowly getting adopted by the wider community. The one place that is having trouble adopting it is the SciPy community, largely because we have not yet adopted rich semantics for numpy arrays.

# Breakpoints! (3.7)
You don't need to do `import pdb; pdb.set_trace()` as your way of setting a breakpoint. What is really nice is that the `breakpoint()` function is actually extensible, so tools like Spyder and Jupyter can have integrated debuggers start up for you!

In [ ]:
def foo():
    a = 1
    breakpoint()
    b = 2
    return a + b

In [ ]:
foo()

# Walrus Operator `:=` (3.8)
In python 3.8, you can use a special kind of assignment operator that works a lot like C/C++'s regular assignment operator for those who are familiar with that. Essentially, in Python, the regular assignment operator can only be used by itself. You can't have it be in an if clause, or a list comprehension, or some other construct. But, in py3.8, the `:=` was introduced to allow those situations to be possible.

A simple example:

In [ ]:
foo = "bar"
print(foo)

Can now be expressed as:

In [ ]:
print(foo := "baz")

Now, where could this be useful, you ask? I find this particularly useful for conditional list comprehensions and generators:

In [ ]:
def some_expensive_func(a, b):
    return a + b

Suppose you need a list of values from this function, but only those greater than 15.

In [ ]:
[some_expensive_func(x1, x2)
 for x1, x2 in zip(range(10), range(20, 0, -2))
 if some_expensive_func(x1, x2) > 15]

Which means you have to compute it twice for each iteration! There are a few other alternative approaches to avoid doing twice the number of function calls, but none of them are very clean or readable. But with a walrus operator, you can do this:

In [ ]:
[val
 for x1, x2 in zip(range(10), range(20, 0, -2))
 if (val := some_expensive_func(x1, x2)) > 15]

Be superfluous with your parentheses. The above won't work without the parens around the assignment operation.

WARNING: While it isn't a syntax error to use this assignment operation just about anywhere, it may still be logically incorrect:

In [ ]:
[(val := some_expensive_func(x1, x2))
 for x1, x2 in zip(range(10), range(20, 0, -2))
 if val > 15]

# Datetime changes
In Py3k, a `datetime.timezone` submodule was created, along with a `utc` timezone object. So, from now on, avoid using `datetime.utcnow()` and `datetime.utcfromtimestamp()`. They are a bit misleading because they will produce naive datetime objects. It is now very easy to create a UTC-aware datetime object that will always do the right thing no matter how it is used:

In [ ]:
from datetime import datetime, timezone
dt_now = datetime.now(tz=timezone.utc)
print(dt_now)
dt_ts = datetime.fromtimestamp(1571595618, timezone.utc)
print(dt_ts)
dt = datetime(2020, 6, 4, 10, 30, tzinfo=timezone.utc)
print(dt)

This is further complicated by the fact that in Py3k (3.6, I think), some operations on naive datetime objects will now implicitly assume that the naive datetime object represents a time in your system's timezone (previously, they would raise an error). So, it may now be better practice to always specify the UTC timezone when we know that we are referencing a time in the UTC timezone, rather than ignoring the timezone altogether.

# Function argument enhancements
Traditionally, arguments to a function can be thought of as positional and keyword arguments:

In [ ]:
def foo(a, b=20):
    return a * b

In `foo()`, both `a` and `b` can be supplied via positional arguments:

`foo(1, 2)`

or via keyword (named) arguments:

`foo(b=2, a=1)`

or a careful mix of the two:

`foo(1, b=2)`

And this has served the Python community well for many years and is certainly better than function argument handling in other languages. But, for those who maintain long-lived libraries, this makes API changes tricky because subtle changes may accidentally break people code. So, in Python 3.0, keyword-only arguments were introduced, and in Python 3.8, positional-only arguments were introduced. All this mean is that there is now a way to specify that particular arguments can only be supplied positionally or via keyword.

## Keyword-only arguments (3.0)
Any arguments with a default value (or `**kwargs`) that come after the `*` entity are considered to be "keyword-only" arguments. Note that arguments without a default value can still come after the `*`, which makes them required keyword arguments.

In [ ]:
def foo(a, *, b=20):
    return a * b

Now, if you want to supply the `b` argument, you _have_ to name it:

In [ ]:
foo(1, b=2)

## Positional-only arguments (3.8)
Any positional arguments that come before the `/` entity are considered "positional-only" arguments. Note that arguments with default values can still come before the `/`, which makes them optional positional arguments.

In [ ]:
def foo(a, /, b=20):
    return a * b

Now, `a` is only known as `a` within the function. It cannot be referenced as such outside the function. It is only known as "the first argument".

In [ ]:
foo(a=1, b=2)

# Matrix multiplication operator `@` (3.5)

In [ ]:
import numpy as np
a = np.array([1.3, 1.5, 1.7])
b = np.array([[2.5, 3.1], [0.2, 0.3], [1.7, -2.0]])

Traditionally, matrix multiplication was done like so:

In [ ]:
np.matmul(a, b)

or

In [ ]:
a.dot(b)

But now you can do:

In [ ]:
a @ b

# Unpacking Fun!
Unpacking is the term used for taking the parts of a collection and assigning them to other variables. We have encountered unpacking before in various forms:

In [ ]:
def foo(a, b=20):
    return a * b

In [ ]:
baz = {'b': 14, 'a': 1}
foo(**baz)

In [ ]:
bar = [10, 15]
foo(*bar)

In [ ]:
a, b = bar
print(a, b)

But now, we can do more!

## Sequence unpacking - Basic (3.0)

In [ ]:
bar = [10, 11, 12, 13, 14, 15]
a, b, *c = bar
print(a)
print(b)
print(c)

In [ ]:
a, *b, c = bar
print(a)
print(b)
print(c)

In [ ]:
*_, a, b = bar
print(a)
print(b)

In [ ]:
ranges = [range(3), range(4)]
for a, *b in ranges:
    print(a, b)

## Sequence unpacking - Additional (3.5)
We also now have new ways to build a list:

In [ ]:
a = [1, 2, 3]
b = [7, 6, 5, 4]
c = [*a, *b]
d = [*b, 10, 11, *a]
print(c)
print(d)

## Set unpacking (3.5)
We can do something similar for sets. It doesn't matter whether the source is a list or a set.

In [ ]:
a = [1, 2, 3, 4, 5]
b = {4, 5, 6, 7}
c = {*a, *b}
d = {*b, 10, 14, 4, *a}
print(c)
print(d)

## Dictionary unpacking (3.5)
Similarly to how we can now create a list using `*` unpacking, we can also create a dictionary using `**` unpacking:

In [ ]:
a = {'foo': 1, 'bar': 2}
b = {'bar': 3, 'baz': 4}
c = {**a, **b}
d = {**a, 'foo': 10, 'baz': 14, **b}
print(c)
print(d)

Notice that in `c`, `'bar'` got the value from `b` because it was unpacked after `a`. And in `d`, `'foo'` got the value of `10` because it was specified after the unpacking of `a`, but `'baz'` was `4` because `b` was unpacked after the specification of `'baz'`.

Also notice, the keys in the dictionaries (as of python 3.5) are retaining their "insertion" order. In Python 2.x, dictionaries had arbitrary order (but were the same from run-to-run if built identically).